In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
import numpy as np
from PIL import Image, ImageFilter
import io
import random
from skimage import data

## Preproccessing 
This includes augmentations, smash and reconstruction of the image, and  high pass filters.

In [3]:
## Data Augmentation


In [9]:


# Define the image processing functions
def jpeg_compression(img, quality_range=(70, 100)):
    if random.random() < 0.1:  # 10% probability
        quality = random.randint(*quality_range)
        img_bytes = io.BytesIO()
        img.save(img_bytes, format='JPEG', quality=quality)
        img = Image.open(img_bytes)
    return img

def gaussian_blur(img, sigma_range=(0, 1)):
    if random.random() < 0.1:  # 10% probability
        sigma = random.uniform(*sigma_range)
        img = img.filter(ImageFilter.GaussianBlur(sigma))
    return img

def downsampling(img, scale_range=(0.25, 0.5)):
    if random.random() < 0.1:  # 10% probability
        scale = random.uniform(*scale_range)
        smaller_img = img.resize((int(img.width * scale), int(img.height * scale)), Image.BICUBIC)
        img = smaller_img.resize((img.width, img.height), Image.BICUBIC)
    return img

# Load a sample image and convert it to a PIL Image
numpy_image = data.astronaut()
img = Image.fromarray(numpy_image)

# Apply the image processing functions
img_compressed = jpeg_compression(img)
img_blurred = gaussian_blur(img)
img_downsampled = downsampling(img)

# Display the processed images to verify the changes
img_compressed.show()
img_blurred.show()
img_downsampled.show()


## Deep Classifier

The following deep classifier has the following layers:
| **Type**    |**Kernel  num**| **With BN** | **Activation** |
| ------------| ------------  | -------     | ----------     |
| Convo.      | 32            | TRUE        | ReLU           |
| Convo.      | 32            | TRUE        | ReLU           |
| Convo.      | 32            | TRUE        | ReLU           |
| Convo.      | 32            | TRUE        | ReLU           |
| Avg Pooling | None          | None        | None           |
| Convo.      | 32            | TRUE        | ReLU           |
| Convo.      | 32            | TRUE        | ReLU           |
| Avg Pooling | None          | None        | None           |
| Convo.      | 32            | TRUE        | ReLU           |
| Convo.      | 32            | TRUE        | ReLU           |
| Avg Pooling | None          | None        | None           |
| Convo.      | 32            | TRUE        | ReLU           |
| Convo.      | 32            | TRUE        | ReLU           |
| AdpAvgPool  | None          | None        | None           |
| Flatten     | None          | None        | None           |
| FC          | None          | FALSE       | None           |

Source: https://arxiv.org/pdf/2311.12397.pdf (page 12)

In [ ]:
class DeepClassifier(nn.Module):
    def __init__(self, num_classes= 1): 
        super(DeepClassifier, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(32)
        self.conv4 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(32)
        self.avg_pool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv5 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(32)
        self.conv6 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(32)
        self.conv7 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn7 = nn.BatchNorm2d(32)
        self.conv8 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn8 = nn.BatchNorm2d(32)
        self.conv9 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn9 = nn.BatchNorm2d(32)
        self.conv10 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn10 = nn.BatchNorm2d(32)
        self.adaptive_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.avg_pool(x)
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.avg_pool(x)
        x = F.relu(self.bn7(self.conv7(x)))
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.avg_pool(x)
        x = F.relu(self.bn9(self.conv9(x)))
        x = F.relu(self.bn10(self.conv10(x)))
        x = self.adaptive_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


model = DeepClassifier()


example_input = torch.rand(4, 3, 64, 64)

output = model(example_input)
print(output.shape)  

torch.Size([4, 1])
